# 추천 시스템 (Recommender Systems)

* 추천 시스템은 크게 두가지로 구분 가능
  * 컨텐츠 기반 필터링 (content-based filtering)
  * 협업 필터링 (collaborative filtering)
* 두가지를 조합한 hybrid 방식도 가능
* 컨텐츠 기반 필터링은 지금까지 사용자의 이전 행동과 명시적 피드백을 통해 사용자가 좋아하는 것과 유사한 항목을 추천
* 협업 필터링은 사용자와 항목간의 유사성을 동시에 사용해 추천

## Surprise

* 추천 시스템 개발을 위한 라이브러리
* 다양한 모델과 데이터 제공
* scikit-learn과 유사한 사용 방법

간단한 surprise 실습

In [1]:
from surprise import SVD, Dataset
from surprise.model_selection import cross_validate

In [2]:
data = Dataset.load_builtin('ml-100k', prompt=False)
data.raw_ratings[:10]

[('196', '242', 3.0, '881250949'),
 ('186', '302', 3.0, '891717742'),
 ('22', '377', 1.0, '878887116'),
 ('244', '51', 2.0, '880606923'),
 ('166', '346', 1.0, '886397596'),
 ('298', '474', 4.0, '884182806'),
 ('115', '265', 2.0, '881171488'),
 ('253', '465', 5.0, '891628467'),
 ('305', '451', 3.0, '886324817'),
 ('6', '86', 3.0, '883603013')]

In [4]:
import pandas as pd
df = pd.DataFrame(data.raw_ratings, columns=['uid','mid','rating','id'])
df.shape

(100000, 4)

In [5]:
model = SVD()       # Singular Value Decomposition

In [7]:
%time cross_validate(model, data, measures=['rmse','mae'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9386  0.9455  0.9327  0.9378  0.9313  0.9372  0.0050  
MAE (testset)     0.7413  0.7456  0.7341  0.7399  0.7316  0.7385  0.0050  
Fit time          6.45    6.92    7.47    7.84    6.92    7.12    0.48    
Test time         0.20    0.28    0.41    0.20    0.28    0.27    0.08    
Wall time: 38.3 s


{'test_rmse': array([0.93856689, 0.94554641, 0.93270293, 0.93779714, 0.93132912]),
 'test_mae': array([0.74127728, 0.74559045, 0.73413281, 0.73990652, 0.73160301]),
 'fit_time': (6.452168226242065,
  6.9198033809661865,
  7.468023300170898,
  7.836069107055664,
  6.918673038482666),
 'test_time': (0.19860267639160156,
  0.28091931343078613,
  0.40836524963378906,
  0.1998913288116455,
  0.28005456924438477)}

## 컨텐츠 기반 필터링 (Content-based Filtering)

* 컨텐츠 기반 필터링은 이전의 행동과 명시적 피드백을 통해 좋아하는 것과 유사한 항목을 추천
  * ex) 내가 지금 까지 시청한 영화 목록과 다른 사용자의 시청 목록을 비교해 나와 비슷한 취향의 사용자가 시청한 영화를 추천
* 유사도를 기반으로 추천
* 컨텐츠 기반 필터링은 다음과 같은 장단점이 있다.
  * 장점
    * 많은 수의 사용자를 대상으로 쉽게 확장 가능
    * 사용자가 관심을 갖지 않던 상품 추천 가능
  * 단점
    * 입력 특성을 직접 설계해야 하기 때문에 많은 도메인 지식이 필요
    * 사용자의 기존 관심사항을 기반으로만 추천 가능

In [8]:
import numpy as np
from surprise import Dataset

* 나와 가장 높은 유사도를 가진 사용자의 시청 목록을 추천

In [10]:
data = Dataset.load_builtin('ml-100k', prompt=False)
raw_data = np.array(data.raw_ratings, dtype=int)
raw_data[:, 0:2] -= 1       # user-id, movie-id를 0부터 시작하게 만듦
raw_data[:10]

array([[      195,       241,         3, 881250949],
       [      185,       301,         3, 891717742],
       [       21,       376,         1, 878887116],
       [      243,        50,         2, 880606923],
       [      165,       345,         1, 886397596],
       [      297,       473,         4, 884182806],
       [      114,       264,         2, 881171488],
       [      252,       464,         5, 891628467],
       [      304,       450,         3, 886324817],
       [        5,        85,         3, 883603013]])

In [11]:
n_users = np.max(raw_data[:, 0])
n_movies = np.max(raw_data[:, 1])
shape = (n_users+1, n_movies+1)
shape

(943, 1682)

In [17]:
# 사용자가 본 영화를 표시하는 행렬을 구한다
adj_matrix = np.zeros(shape, dtype=int)
for user_id, movie_id, _, _ in raw_data:
    adj_matrix[user_id, movie_id] = 1
adj_matrix[:5]

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0]])

- 이진 벡터의 내적을 통해 다른 사용자들과의 유사도 구하기

In [18]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(adj_matrix):
    if my_id != user_id:
        similarity = np.dot(my_vector, user_vector)
        if similarity > best_match:
            best_match = similarity
            best_match_id = user_id
            best_match_vector = user_vector

best_match, best_match_id

(183, 275)

In [20]:
# 내가 보지 않은 영화중에서 best_match 사용자가 본 영화 
recommend_list = []
for i, (my_view, best_match_view) in enumerate(zip(my_vector, best_match_vector)):
    if my_view == 0 and best_match_view == 1:
        recommend_list.append(i)
len(recommend_list), recommend_list[:10]

(335, [272, 273, 275, 280, 281, 283, 287, 288, 289, 290])

* 유클리드 거리를 사용해 추천
$$euclidean = \sqrt{\sum_{d=1}^{D}(A_i - B_i)^2}$$
* 거리가 가까울 수록(값이 작을 수록) 나와 유사한 사용자

In [22]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = 9999, -1, []

for user_id, user_vector in enumerate(adj_matrix):
    if my_id != user_id:
        euc_dist = np.sqrt(np.sum(np.square(my_vector - user_vector)))
        if euc_dist < best_match:
            best_match = euc_dist
            best_match_id = user_id
            best_match_vector = user_vector

best_match, best_match_id

(14.832396974191326, 737)

In [23]:
# 내가 보지 않은 영화중에서 best_match 사용자가 본 영화 
recommend_list = []
for i, (my_view, best_match_view) in enumerate(zip(my_vector, best_match_vector)):
    if my_view == 0 and best_match_view == 1:
        recommend_list.append(i)
len(recommend_list), recommend_list[:10]

(48, [297, 312, 317, 342, 356, 366, 379, 384, 392, 402])

* 코사인 유사도를 사용해 추천

\begin{equation}
cos \theta = \frac{A \cdot B}{||A|| \times ||B||}
\end{equation}
* 두 벡터가 이루고 있는 각을 계산

In [24]:
def cos_sim(v1, v2):
    norm1 = np.sqrt(np.sum(np.square(v1)))
    norm2 = np.sqrt(np.sum(np.square(v2)))
    return np.dot(v1, v2) / (norm1 * norm2)

In [25]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(adj_matrix):
    if my_id != user_id:
        cos_similarity = cos_sim(my_vector, user_vector)
        if cos_similarity > best_match:
            best_match = cos_similarity
            best_match_id = user_id
            best_match_vector = user_vector

best_match, best_match_id

(0.5278586163659506, 915)

In [26]:
# 내가 보지 않은 영화중에서 best_match 사용자가 본 영화 
recommend_list = []
for i, (my_view, best_match_view) in enumerate(zip(my_vector, best_match_vector)):
    if my_view == 0 and best_match_view >= 1:
        recommend_list.append(i)
len(recommend_list), recommend_list[:10]

(162, [272, 275, 279, 280, 283, 285, 289, 294, 297, 316])

기존 방법에 명시적 피드백(사용자가 평가한 영화 점수)을 추가해 실험

In [28]:
# 사용자가 본 영화의 평점 행렬을 구한다
adj_matrix = np.zeros(shape, dtype=int)
for user_id, movie_id, rating, _ in raw_data:
    adj_matrix[user_id, movie_id] = rating
adj_matrix[:5]

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [4, 3, 0, ..., 0, 0, 0]])

In [29]:
# 유클리드 거리
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = 9999, -1, []

for user_id, user_vector in enumerate(adj_matrix):
    if my_id != user_id:
        euc_dist = np.sqrt(np.sum(np.square(my_vector - user_vector)))
        if euc_dist < best_match:
            best_match = euc_dist
            best_match_id = user_id
            best_match_vector = user_vector

best_match, best_match_id

(55.06359959174482, 737)

In [ ]:
# 코사인 유사도
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(adj_matrix):
    if my_id != user_id:
        cos_similarity = cos_sim(my_vector, user_vector)
        if cos_similarity > best_match:
            best_match = cos_similarity
            best_match_id = user_id
            best_match_vector = user_vector

best_match, best_match_id

Best Match: 0.569065731527988, Best Match ID: 915


## 협업 필터링(Collaborative Filtering)

* 사용자와 항목의 유사성을 동시에 고려해 추천
* 기존에 내 관심사가 아닌 항목이라도 추천 가능
* 자동으로 임베딩 학습 가능
* 협업 필터링은 다음과 같은 장단점을 갖고 있다.
  * 장점
    * 자동으로 임베딩을 학습하기 때문에 도메인 지식이 필요 없다.
    * 기존의 관심사가 아니더라도 추천 가능
  * 단점
    * 학습 과정에 나오지 않은 항목은 임베딩을 만들 수 없음
    * 추가 특성을 사용하기 어려움

In [30]:
from surprise import KNNBasic, SVD, SVDpp, NMF
from surprise import Dataset
from surprise.model_selection import cross_validate

In [31]:
data = Dataset.load_builtin('ml-100k', prompt=False)

* KNN(K-Nearest Neighbor)을 사용한 협업 필터링

In [32]:
model = KNNBasic()
cross_validate(model, data, measures=['rmse','mae'], cv=5, n_jobs=-1, verbose=True)

Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9755  0.9795  0.9775  0.9814  0.9810  0.9790  0.0022  
MAE (testset)     0.7709  0.7739  0.7749  0.7739  0.7729  0.7733  0.0014  
Fit time          0.99    1.08    1.08    1.11    1.07    1.06    0.04    
Test time         6.64    6.46    6.64    6.42    6.43    6.52    0.10    
Wall time: 10.8 s


{'test_rmse': array([0.97551365, 0.979529  , 0.97754382, 0.98137088, 0.98104487]),
 'test_mae': array([0.77091698, 0.77392485, 0.77492405, 0.7738629 , 0.77285361]),
 'fit_time': (0.9919707775115967,
  1.0756721496582031,
  1.080507516860962,
  1.1068511009216309,
  1.0686802864074707),
 'test_time': (6.639443874359131,
  6.464093923568726,
  6.638692140579224,
  6.417373418807983,
  6.433128833770752)}

* SVD(Singular Value Decomposition)를 사용한 협업 필터링

In [33]:
model = SVD()
cross_validate(model, data, measures=['rmse','mae'], cv=5, n_jobs=-1, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9384  0.9355  0.9293  0.9413  0.9357  0.9360  0.0040  
MAE (testset)     0.7423  0.7383  0.7328  0.7399  0.7381  0.7383  0.0031  
Fit time          10.95   10.88   10.95   10.79   10.82   10.88   0.06    
Test time         0.40    0.39    0.39    0.38    0.43    0.40    0.02    


{'test_rmse': array([0.93836791, 0.93545566, 0.92933402, 0.94131684, 0.93568757]),
 'test_mae': array([0.74229917, 0.73829655, 0.73284563, 0.73992791, 0.73807113]),
 'fit_time': (10.95334506034851,
  10.876849889755249,
  10.945230722427368,
  10.79177451133728,
  10.822291612625122),
 'test_time': (0.3994007110595703,
  0.3941175937652588,
  0.38959813117980957,
  0.38437342643737793,
  0.43326711654663086)}

* NMF(Non-negative Matrix Factorization)를 사용한 협업 필터링

In [34]:
model = NMF()
cross_validate(model, data, measures=['rmse','mae'], cv=5, n_jobs=-1, verbose=True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9648  0.9575  0.9627  0.9744  0.9528  0.9625  0.0073  
MAE (testset)     0.7587  0.7537  0.7552  0.7642  0.7489  0.7561  0.0051  
Fit time          10.59   10.62   10.75   10.81   10.89   10.73   0.11    
Test time         0.32    0.35    0.34    0.30    0.28    0.32    0.03    


{'test_rmse': array([0.96483812, 0.95753746, 0.96272712, 0.97444842, 0.95278709]),
 'test_mae': array([0.75869207, 0.753654  , 0.75516034, 0.76417015, 0.74892032]),
 'fit_time': (10.593607902526855,
  10.624523162841797,
  10.753577709197998,
  10.81222152709961,
  10.886921405792236),
 'test_time': (0.31989002227783203,
  0.3538475036621094,
  0.33539581298828125,
  0.3011023998260498,
  0.277878999710083)}

* SVD++를 사용한 협업 필터링

In [35]:
model = SVDpp()
cross_validate(model, data, measures=['rmse','mae'], cv=5, n_jobs=-1, verbose=True)

Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9217  0.9229  0.9092  0.9191  0.9198  0.9186  0.0049  
MAE (testset)     0.7256  0.7244  0.7131  0.7180  0.7227  0.7208  0.0046  
Fit time          378.98  378.01  377.17  376.15  375.62  377.18  1.22    
Test time         4.31    4.74    4.92    4.93    4.88    4.76    0.23    


{'test_rmse': array([0.92171623, 0.92293947, 0.90918642, 0.91914275, 0.91983392]),
 'test_mae': array([0.72559001, 0.72444707, 0.71306976, 0.71799834, 0.72272576]),
 'fit_time': (378.98021483421326,
  378.00732374191284,
  377.168568611145,
  376.14919328689575,
  375.61803245544434),
 'test_time': (4.310297727584839,
  4.740556955337524,
  4.918924808502197,
  4.931668043136597,
  4.882288455963135)}

## 하이브리드(Hybrid)

* 컨텐츠 기반 필터링과 협업 필터링을 조합한 방식
* 많은 하이브리드 방식이 존재
* 실습에서는 협업 필터링으로 임베딩을 학습하고 컨텐츠 기반 필터링으로 유사도 기반 추천을 수행하는 추천 엔진 개발

In [36]:
import numpy as np
from sklearn.decomposition import randomized_svd, non_negative_factorization
from surprise import Dataset

In [37]:
data = Dataset.load_builtin('ml-100k', prompt=False)
raw_data = np.array(data.raw_ratings, dtype=int)
raw_data[:, 0:2] -= 1       # user-id, movie-id를 0부터 시작하게 만듦
raw_data[:5]

array([[      195,       241,         3, 881250949],
       [      185,       301,         3, 891717742],
       [       21,       376,         1, 878887116],
       [      243,        50,         2, 880606923],
       [      165,       345,         1, 886397596]])

In [38]:
n_users = np.max(raw_data[:, 0])
n_movies = np.max(raw_data[:, 1])
shape = (n_users+1, n_movies+1)
shape

(943, 1682)

In [39]:
# 사용자가 본 영화의 평점 행렬을 구한다
adj_matrix = np.zeros(shape, dtype=int)
for user_id, movie_id, rating, _ in raw_data:
    adj_matrix[user_id, movie_id] = rating
adj_matrix[:5]

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [4, 3, 0, ..., 0, 0, 0]])

In [53]:
U, S, V = randomized_svd(adj_matrix, n_components=2)
S = np.diag(S)
U.shape, S.shape, V.shape

((943, 2), (2, 2), (2, 1682))

In [54]:
result = np.matmul(np.matmul(U, S), V)
print(result.shape)
result[:3]

(943, 1682)


array([[ 3.91732663e+00,  1.47276644e+00,  7.98261988e-01, ...,
         6.24907189e-04,  1.41100852e-02,  1.36545878e-02],
       [ 1.85777226e+00,  3.96191175e-01,  5.05705740e-01, ...,
         5.38862978e-03,  1.77237914e-03,  5.26968095e-04],
       [ 8.94989517e-01,  1.71578497e-01,  2.51738682e-01, ...,
         2.92094923e-03,  5.39937171e-04, -1.25733753e-04]])

* 사용자 기반 추천
* 나와 비슷한 취향을 가진 다른 사용자의 행동을 추천
* 사용자 특징 벡터의 유사도 사용

In [42]:
def compute_cos_sim(v1, v2):
    norm1 = np.sqrt(np.sum(np.square(v1)))
    norm2 = np.sqrt(np.sum(np.square(v2)))
    return np.dot(v1, v2) / (norm1 * norm2)

In [55]:
my_id, my_vector = 0, U[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(U):
    if my_id != user_id:
        cos_similarity = compute_cos_sim(my_vector, user_vector)
        if cos_similarity > best_match:
            best_match = cos_similarity
            best_match_id = user_id
            best_match_vector = user_vector

best_match, best_match_id

(0.9999942295956322, 235)

In [56]:
recommend_list = []
for i, (my_view, best_match_view) in enumerate(zip(adj_matrix[my_id], adj_matrix[best_match_id])):
    if my_view == 0 and best_match_view >= 1:
        recommend_list.append(i)
len(recommend_list), recommend_list[:10]

(69, [272, 273, 274, 281, 285, 288, 293, 297, 303, 306])

* 항목 기반 추천
* 내가 본 항목과 비슷한 항목을 추천
* 항목 특징 벡터의 유사도 사용

In [57]:
my_id, my_vector = 0, V.T[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(V.T):
    if my_id != user_id:
        cos_similarity = compute_cos_sim(my_vector, user_vector)
        if cos_similarity > best_match:
            best_match = cos_similarity
            best_match_id = user_id
            best_match_vector = user_vector

best_match, best_match_id

(0.9999999951364141, 1287)

In [58]:
recommend_list = []
for i, _ in enumerate(adj_matrix):
    if adj_matrix[i, my_id] >= 1:
        recommend_list.append(i)
len(recommend_list), recommend_list[:10]

(452, [0, 1, 4, 5, 9, 12, 14, 15, 16, 17])

* 비음수 행렬 분해를 사용한 하이브리드 추천

In [47]:
adj_matrix[:5]

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [4, 3, 0, ..., 0, 0, 0]])

In [49]:
import warnings
warnings.filterwarnings('ignore')
A, B, iter = non_negative_factorization(adj_matrix, n_components=2)

In [50]:
result = np.matmul(A, B)
print(result.shape)
result[:3]

(943, 1682)


array([[3.71108858e+00, 1.48454102e+00, 7.39535363e-01, ...,
        3.64490531e-03, 1.45506510e-02, 1.44110916e-02],
       [2.11724416e+00, 2.37338725e-01, 5.51650703e-01, ...,
        4.76092332e-03, 3.03585166e-05, 0.00000000e+00],
       [9.85272360e-01, 1.10447010e-01, 2.56713988e-01, ...,
        2.21552443e-03, 1.41275191e-05, 0.00000000e+00]])

* 사용자 기반 추천

In [60]:
my_id, my_vector = 0, A[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(A):
    if my_id != user_id:
        cos_similarity = compute_cos_sim(my_vector, user_vector)
        if cos_similarity > best_match:
            best_match = cos_similarity
            best_match_id = user_id
            best_match_vector = user_vector

best_match, best_match_id

(0.9999996856299366, 314)

In [61]:
recommend_list = []
for i, (my_view, best_match_view) in enumerate(zip(adj_matrix[my_id], adj_matrix[best_match_id])):
    if my_view == 0 and best_match_view >= 1:
        recommend_list.append(i)
len(recommend_list), recommend_list[:10]

(41, [272, 275, 284, 285, 287, 300, 301, 302, 304, 317])

* 항목 기반 추천

In [62]:
my_id, my_vector = 0, B.T[0]  # 기준 항목이 0번 이라고 가정
best_match, best_match_id, best_match_vector = -1, -1, []

for item_id, item_vector in enumerate(B.T):
  if my_id != item_id:
    cos_similarity = compute_cos_sim(my_vector, item_vector)
    if cos_similarity > best_match:
      best_match = cos_similarity
      best_match_id = item_id
      best_match_vector = item_vector

best_match, best_match_id

(0.9999999946284729, 255)

In [63]:
recommend_list = []
for i, _ in enumerate(adj_matrix):
    if adj_matrix[i, my_id] >= 1:
        recommend_list.append(i)
len(recommend_list), recommend_list[:10]

(452, [0, 1, 4, 5, 9, 12, 14, 15, 16, 17])